In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf

In [2]:
# a = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True, infer_shape=False)
# a.write(0, [1., 0.])
# a.write(1, [[0., 1.], [1., 2.]])
# b = a.concat()
# tf.reduce_mean(b, 0)

a = tf.constant([[[0, 1, 2]]])

tf.transpose(a, [0, 1, 2])

2023-12-30 04:46:15.174467: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-30 04:46:15.174525: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-12-30 04:46:15.175541: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(1, 1, 3), dtype=int32, numpy=array([[[0, 1, 2]]], dtype=int32)>

In [42]:
flag = tf.cast(tf.reduce_sum(tf.zeros((16,16, 16))), tf.bool)
flag

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [44]:
v = tf.Variable(1)
@tf.function
def f(x):
    ta = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
    for i in tf.range(x):
        tf.print("{}".format(i))
        v.assign_add(i)
        ta = ta.write(i, v)
    return ta.stack()

f(5)

Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/Placeholder:0", shape=(), dtype=int32)


<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 1,  2,  4,  7, 11], dtype=int32)>